In [1]:
import os
import ctools
import numpy as np
import random
import math
from astropy.io import fits
from lxml import etree

models=os.path.join("..", "models")
data=os.path.join("..", "data")

In [2]:
#creation of data directory
if not os.path.isdir(data):
    os.mkdir(data)
    
#creation of directories for all simulations 
#one source group3 simulation
sourceSimulations=os.path.join(data, "sourceSimulations")
if not os.path.isdir(sourceSimulations):
    os.mkdir(sourceSimulations)
#background simulation
backgroundSimulations=os.path.join(data, "backgroundSimulations")
if not os.path.isdir(backgroundSimulations):
    os.mkdir(backgroundSimulations)
#two sources simulation
twoSourcesSimulations=os.path.join(data, "twoSourcesSimulations")
if not os.path.isdir(twoSourcesSimulations):
    os.mkdir(twoSourcesSimulations)
    
#creation for directories for all models 
#one source group3 model
sourceModels=os.path.join(models, "sourceModels")    
if not os.path.isdir(sourceModels):
    os.mkdir(sourceModels)
#background model
backgroundModels=os.path.join("..", "models", "backgroundModels")
if not os.path.isdir(backgroundModels):
    os.mkdir(backgroundModels)
#two sources model
twoSourcesModels=os.path.join("..", "models", "twoSourcesModels")
if not os.path.isdir(twoSourcesModels):
    os.mkdir(twoSourcesModels)

In [3]:
def run_ctobssim(model, output,pointing = (83.6331, 22.5145), energy=(0.03, 5), time=(0, 1200), fov=5, caldb='prod3b-v2', irf='South_z40_0.5h', seed=1):
    sim = ctools.ctobssim()
    sim["inmodel"] = model #inserisce un modello pre-esistente all'interno del tool di generazione mappe
    sim["outevents"] = output #estrae una mappa creata in estensione .fits
    sim["caldb"] = caldb #calibration database, inserito nella fase di configurazione come zip
    sim["irf"] = irf #Instrumental response function, mappa tra flusso di fotoni in ingresso e gli eventi detected
    sim["ra"] = pointing[0] #Right Ascension of CTA pointing (dove punta nello spazio il telescopio)
    sim["dec"] = pointing[1] #Declination of CTA pointing (dove punta nello spazio il telescopio)
    sim["rad"] = fov #field of view, simulation cone radius, 5 rad mi pare un botto
    sim["tmin"] = time[0] #start time
    sim["tmax"] = time[1] #stop time of the exposure
    sim["emin"] = energy[0] #Lower energy limit of simulated events (in TeV).
    sim["emax"] = energy[1] #Upper energy limit of simulated events (in TeV).
    sim["seed"] = seed #random number generator for the simulations with a seed value provided by the hidden seed parameter
    sim.execute()
    # if you want to run in memory:
    #sim.run()
    
def run_skymap(obs, output, energy=(0.03, 5), roi=5, caldb='prod3b-v2', irf='North_z20_0.5h', wbin=0.02):
    nbin = int(roi*2/wbin)
    skymap = ctools.ctskymap()
    skymap['inobs'] = obs
    skymap['outmap'] = output
    skymap['irf'] = irf 
    skymap['caldb'] = caldb #calibration database, inserito nella fase di configurazione come zip
    skymap['emin'] = energy[0]
    skymap['emax'] = energy[1]
    skymap['usepnt'] = True #use pointing
    skymap['nxpix'] = nbin #Size of the Right Ascension / Galactic longitude axis (in pixels).
    skymap['nypix'] = nbin #Size of the Declination / Galactic latitude axis (in pixels).
    skymap['binsz'] = wbin #Pixel size (in degrees/pixel).
    skymap['bkgsubtract'] = 'IRF' #da testare 'NONE' o 'IRF'
    skymap.execute()
    # if you want to run in memory:
    #sim.run()
    
def truncate(num, n):
    integer = int(num * (10**n))/(10**n)
    return float(integer)

In [4]:
def modify_source_position(ra, dec, index):
    tree = etree.parse(os.path.join(models, "group3.xml"))
    root = tree.getroot()
    Ra = root[0][1][0]
    Ra.set("value", str(ra))
    Dec = root[0][1][1]
    Dec.set("value", str(dec))
    et = etree.ElementTree(root)
    name = os.path.join(sourceModels, "group3_" + str(index) + ".xml") 
    et.write(name, pretty_print=True)
        
def modify_two_source_position(ra, dec, index, ra2, dec2):
    #find the second point in the xml file 
    tree = etree.parse(os.path.join(models, "crab2sources.xml"))
    root = tree.getroot()
    Ra = root[0][1][0]
    Ra.set("value", str(ra))
    Dec = root[0][1][1]
    Dec.set("value", str(dec))
    #modify the value and write the model
    Ra2 = root[1][1][0]
    Ra2.set("value", str(ra2))
    Dec2 = root[1][1][1]
    Dec2.set("value", str(dec2))
    et = etree.ElementTree(root)
    name = os.path.join(twoSourcesModels, "crab2sources_" + str(index) + ".xml")
    et.write(name, pretty_print=True)
        

In [5]:
def create_new_model(i, pointing, maxAng=4.5, twoPoints="false"):
    
    #random with Module and Angle 
    angle = random.random() *2 *math.pi
    cos = math.cos(angle)
    sin = math.sin(angle)
    mod = random.random()*maxAng

    ra = round(pointing[0] + (sin*mod),4)
    dec = round(pointing[1] + (cos*mod),4)
    
    if(twoPoints == "true"):
        ipo = 0
        while(ipo<2): #to modify ------------------------------------------
            angle = random.random() *2 *math.pi
            cos = math.cos(angle)
            sin = math.sin(angle)
            mod = random.random()*maxAng
            ra2 = round(pointing[0] + (sin*mod),4)
            dec2 = round(pointing[1] + (cos*mod),4)
            ipo = ((ra-ra2)**2 + (dec-dec2)**2)**1/2
        modify_two_source_position(ra,dec,i,ra2,dec2)
        
    #richiamiamo la modifica e creazione di nuovi modelli coi parametri genrati pocanzi
    modify_source_position(ra,dec,i)

## Source simulation and map

In this step we are going to create models and fits map based on the model given (group3.xml)

In [6]:
count=1
pointing = (83.6331, 22.5145)
for i in range(count):
    create_new_model(i,pointing)
    obs = os.path.join(sourceModels, "group3_sim.fits")
    model = os.path.join(sourceModels, "group3_" + str(i) + ".xml") 
    run_ctobssim(model=model, output=obs)
    run_skymap(obs=obs, output=os.path.join(sourceSimulations, "group3_sky_"+ str(i) + ".fits"))
os.remove(obs)

## Background simulation and map

in this step we are going to create background simulation

In [7]:
count=1
pointing = (83.6331, 22.5145)
for i in range(count):
    model = os.path.join(models,"bkg_irf.xml")
    obs = os.path.join(backgroundModels,"bkg_test_sim.fits")
    run_ctobssim(model=model, output=obs,seed= int(random.random()*100))
    run_skymap(obs=obs, output=os.path.join(backgroundSimulations,"bkg_test_sky_" + str(i) + ".fits"))
os.remove(obs)

## Two sources simulation and map
In this step we are going to create models and fits map based on the model given (crab2sources.xml)

In [8]:
count=1
pointing = (83.6331, 22.5145)
for i in range(count):
    create_new_model(i,pointing,twoPoints="true")
    obs = os.path.join(twoSourcesModels, "crab2sources_sim.fits")
    model = os.path.join(twoSourcesModels, "crab2sources_" + str(i) + ".xml") 
    run_ctobssim(model=model, output=obs)
    run_skymap(obs=obs, output=os.path.join(twoSourcesSimulations, "crab2sources_"+ str(i) + ".fits"))
os.remove(obs)